In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
import torch


In [1]:
from datasets import load_dataset

dataset = load_dataset(
    "empathetic_dialogues"
)

print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 76673
    })
    validation: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 12030
    })
    test: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 10943
    })
})


In [2]:
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer


model = AutoModelForCausalLM.from_pretrained("distilgpt2")

training_args = TrainingArguments(
    output_dir="./empathetic_chatbot",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",
    num_train_epochs=2,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
     report_to=[]  # Disable all reporting integrations including wandb
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


/tmp/ipython-input-3-677815203.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=38338, training_loss=3.011539232961585, metrics={'train_runtime': 4275.8437, 'train_samples_per_second': 35.863, 'train_steps_per_second': 8.966, 'total_flos': 5008601439535104.0, 'train_loss': 3.011539232961585, 'epoch': 2.0})

In [4]:
model.save_pretrained("empathetic_distilgpt2")
tokenizer.save_pretrained("empathetic_distilgpt2")


('empathetic_distilgpt2/tokenizer_config.json',
 'empathetic_distilgpt2/special_tokens_map.json',
 'empathetic_distilgpt2/vocab.json',
 'empathetic_distilgpt2/merges.txt',
 'empathetic_distilgpt2/added_tokens.json',
 'empathetic_distilgpt2/tokenizer.json')

In [10]:
import torch

def chat():
    print("Empathetic Mental Health Chatbot 🤗")
    print("Type 'exit' to end the conversation.\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break

        # Tokenize input
        inputs = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True)

        # Move inputs to the same device as the model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # Generate response
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_length=100,
                pad_token_id=tokenizer.eos_token_id
            )

        response = tokenizer.decode(output[0], skip_special_tokens=True)
        print("Bot:", response.replace(user_input, "").strip())


# Run the chatbot
chat()

Empathetic Mental Health Chatbot 🤗
Type 'exit' to end the conversation.

You: i have done great work
Bot: on my job and i am so happy for you!  I am so happy for you!
You: exit
